## Quiz #0502

### "Human Activity Recognition" 

#### Answer the following questions by providing Python code:
#### Objectives:
- Carry out the EDA.
- Carry out the data pre-processing.
- Optimize and test a predictive model of your choice.

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn import metrics, preprocessing
warnings.filterwarnings(action='ignore')                  # Turn off the warnings.
%matplotlib inline

#### Read in data:
The explanation on the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones).

In [2]:
# Go to the directory where the data file is located. 
# os.chdir(r'~~')                # Please, replace the path with your own. 

In [45]:
df = pd.read_csv('data_human activity recognition.csv', header='infer')

In [46]:
df.shape

(19622, 160)

In [47]:
df.columns

Index(['Unnamed: 0', 'user_name', 'raw_timestamp_part_1',
       'raw_timestamp_part_2', 'cvtd_timestamp', 'new_window', 'num_window',
       'roll_belt', 'pitch_belt', 'yaw_belt',
       ...
       'gyros_forearm_x', 'gyros_forearm_y', 'gyros_forearm_z',
       'accel_forearm_x', 'accel_forearm_y', 'accel_forearm_z',
       'magnet_forearm_x', 'magnet_forearm_y', 'magnet_forearm_z', 'classe'],
      dtype='object', length=160)

1). Carry out the EDA. Check for the missing values. HINT: The response variable is 'classe'.

In [48]:
       # <Your code goes in here>
#Checking for the missing values 
s = df.isnull().sum()
s = s[s>0]
print(s) # We see that there are 100 features with missing values

kurtosis_roll_belt      19216
kurtosis_picth_belt     19216
kurtosis_yaw_belt       19216
skewness_roll_belt      19216
skewness_roll_belt.1    19216
                        ...  
stddev_pitch_forearm    19216
var_pitch_forearm       19216
avg_yaw_forearm         19216
stddev_yaw_forearm      19216
var_yaw_forearm         19216
Length: 100, dtype: int64


2). Get rid of the columns that have more than 97% missing values.

In [49]:
       # <Your code goes in here>
# Finding the columns with more than 97% missing values
n_rows = df.shape[0]
col = list(s[s>(97*n_rows/100)].index) #We get the index of these columns
print("Number of columns to drop :",len(col))
df.drop(columns=col,inplace=True) #Dropping these columns
print('There are :',int((df.isnull().sum().shape)[0]),'remaining columns ')
df.head()

Number of columns to drop : 100
There are : 60 remaining columns 


,Unnamed: 0,user_name,raw_timestamp_part_1,raw_timestamp_part_2,cvtd_timestamp,new_window,num_window,roll_belt,pitch_belt,yaw_belt,total_accel_belt,gyros_belt_x,gyros_belt_y,gyros_belt_z,accel_belt_x,accel_belt_y,accel_belt_z,magnet_belt_x,magnet_belt_y,magnet_belt_z,roll_arm,pitch_arm,yaw_arm,total_accel_arm,gyros_arm_x,gyros_arm_y,gyros_arm_z,accel_arm_x,accel_arm_y,accel_arm_z,magnet_arm_x,magnet_arm_y,magnet_arm_z,roll_dumbbell,pitch_dumbbell,yaw_dumbbell,total_accel_dumbbell,gyros_dumbbell_x,gyros_dumbbell_y,gyros_dumbbell_z,accel_dumbbell_x,accel_dumbbell_y,accel_dumbbell_z,magnet_dumbbell_x,magnet_dumbbell_y,magnet_dumbbell_z,roll_forearm,pitch_forearm,yaw_forearm,total_accel_forearm,gyros_forearm_x,gyros_forearm_y,gyros_forearm_z,accel_forearm_x,accel_forearm_y,accel_forearm_z,magnet_forearm_x,magnet_forearm_y,magnet_forearm_z,classe
0,1,carlitos,1323084231,788290,05/12/2011 11:23,no,11,1.41,8.07,-94.4,3,0.00,0.00,-0.02,-21,4,22,-3,599,-313,-128.0,22.5,-161.0,34,0.00,0.00,-0.02,-288,109,-123,-368,337,516,13.052175,-70.494004,-84.873939,37,0.0,-0.02,0.00,-234,47,-271,-559,293,-65.0,28.4,-63.9,-153.0,36,0.03,0.00,-0.02,192,203,-215,-17,654.0,476.0,A
1,2,carlitos,1323084231,808298,05/12/2011 11:23,no,11,1.41,8.07,-94.4,3,0.02,0.00,-0.02,-22,4,22,-7,608,-311,-128.0,22.5,-161.0,34,0.02,-0.02,-0.02,-290,110,-125,-369,337,513,13.130740,-70.637505,-84.710647,37,0.0,-0.02,0.00,-233,47,-269,-555,296,-64.0,28.3,-63.9,-153.0,36,0.02,0.00,-0.02,192,203,-216,-18,661.0,473.0,A
2,3,carlitos,1323084231,820366,05/12/2011 11:23,no,11,1.42,8.07,-94.4,3,0.00,0.00,-0.02,-20,5,23,-2,600,-305,-128.0,22.5,-161.0,34,0.02,-0.02,-0.02,-289,110,-126,-368,344,513,12.850750,-70.278120,-85.140781,37,0.0,-0.02,0.00,-232,46,-270,-561,298,-63.0,28.3,-63.9,-152.0,36,0.03,-0.02,0.00,196,204,-213,-18,658.0,469.0,A
3,4,carlitos,1323084232,120339,05/12/2011 11:23,no,12,1.48,8.05,-94.4,3,0.02,0.00,-0.03,-22,3,21,-6,604,-310,-128.0,22.1,-161.0,34,0.02,-0.03,0.02,-289,111,-123,-372,344,512,13.431200,-70.393795,-84.873626,37,0.0,-0.02,-0.02,-232,48,-269,-552,303,-60.0,28.1,-63.9,-152.0,36,0.02,-0.02,0.00,189,206,-214,-16,658.0,469.0,A
4,5,carlitos,1323084232,196328,05/12/2011 11:23,no,12,1.48,8.07,-94.4,3,0.02,0.02,-0.02,-21,2,24,-6,600,-302,-128.0,22.1,-161.0,34,0.00,-0.03,0.00,-289,111,-123,-374,337,506,13.378716,-70.428560,-84.853057,37,0.0,-0.02,0.00,-233,48,-270,-554,292,-68.0,28.0,-63.9,-152.0,36,0.02,0.00,-0.02,189,206,-214,-17,655.0,473.0,A


3). Get rid of the unnecessary columns. HINT: Those columns with "time" in the name and those that are obviously unnecessary.

In [50]:
       # <Your code goes in here>
df.drop(columns='Unnamed: 0',inplace=True)
df.drop(columns=df.columns[df.columns.str.contains('time')],inplace=True)

4). Label encode the responde variable. HINT: use preprocessing.LabelEncoder(). 

In [51]:
       # <Your code goes in here>
from sklearn.preprocessing import LabelEncoder
print('Before label encoding :',df['classe'].unique())
le = LabelEncoder()
df["classe"] = le.fit_transform(df["classe"])
print('After label encoding :',df['classe'].unique())

Before label encoding : ['A' 'B' 'C' 'D' 'E']
After label encoding : [0 1 2 3 4]


5). Carry out min-max scaling of the exploratory variables. HINT: use preprocessing.MinMaxScaler().

In [52]:
       # <Your code goes in here>
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
col = list(df.drop('classe',axis=1).select_dtypes(include='number').columns) # selecting the numerical columns  
df[col] = scaler.fit_transform(df[col])

6). Choose an algorithm and carry out the predictive analysis.

- Optimize the hyperparameter(s)
- Calculate the accuracy.
- Aim for upper 90% accuracy.

In [54]:
       # <Your code goes in here>
X=df[col]
Y=df['classe']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

#We choose the random forest algorithm
RFC = RandomForestClassifier()
#Optimizing the hyperparameters using GridSearchCV 
n_estimators = [10 , 50, 100, 300]
max_depth = [5, 8, 15]

param_dict = dict(n_estimators = n_estimators , max_depth=max_depth)

gridF = GridSearchCV(RFC, param_dict, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, Y_train)
Y_pred = gridF.predict(X_test)
print( "Random Forest accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))
print("The best parameters are :",bestF.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:  1.6min finished


Random Forest accuracy : 0.996
The best parameters are : {'max_depth': 15, 'n_estimators': 300}
